# 課題1(Optional) 乗換案内
実装のポイント：cacheのときのように，自分のひとつ前のノードを覚えておいて，後ろから逆順にたどればよい．  
注意すべき点は，次のノードは複数存在するかもしれないが，自分に至るまでの最小ルートは選べることである．  
なので，逆順からたどっている．

## 準備

In [0]:
from google.colab import auth, drive, files, output
drive.mount('/content/drive')

In [0]:
import os
os.chdir("/content/drive/My Drive/STEP2020")

In [0]:
#stationsの読み込み
f1 = open('txt/station/stations.txt')
stations = f1.read()
f1.close()
stations = stations.split("\n")
stations = [stations[i].split('\t')[1] for i in range(len(stations))]
#stations #確認用

In [0]:
#駅名から番号を検索できるようにしておく．
from collections import defaultdict
name2num = defaultdict(int)
for i in range(len(stations)):
    name2num[stations[i]] = i
#name2num #確認用

In [0]:
#edgesの読み込み
f1 = open('txt/station/edges.txt')
edges = f1.read()
f1.close()
edges = edges.split("\n")
edges = tuple([tuple(map(int, edges[i].split('\t'))) for i in range(len(edges))])
#edges #確認用

In [0]:
#隣接グラフの作成
neighbors = [[] for i in range(len(stations))]
for edge in edges:
    neighbors[edge[0]].append([edge[1], edge[2]])
    neighbors[edge[1]].append([edge[0], edge[2]])
#neighbors #確認用

## 最短ルートと所要時間

In [39]:
import heapq
import time
from collections import deque

def dijkstra(dist_from_start, start, goal):
    hq = [] #[スタートから次ノードまでの距離，次のノード，現在のノード]をスタートから次ノードまでの距離の順にソートして持たせる．
    for neighbor in neighbors[start]:
        heapq.heappush(hq, [dist_from_start[start] + neighbor[1], neighbor[0], start])
    while len(hq) > 0:
        node = heapq.heappop(hq)
        if node[1] == goal:
            dist_from_start[node[1]] = node[0]
            prev_nodes[node[1]] = node[2]
            return node[0]
        elif dist_from_start[node[1]] == float('inf'):
            dist_from_start[node[1]] = node[0]
            prev_nodes[node[1]] = node[2]
            for neighbor in neighbors[node[1]]:
                if dist_from_start[neighbor[0]] == float('inf'):
                    heapq.heappush(hq, [dist_from_start[node[1]] + neighbor[1], neighbor[0], node[1]])
    return float('inf')

start = name2num['渋谷']
goal = name2num['上野']
dist_from_start = [float('inf') for i in range(len(stations))]
dist_from_start[start] = 0
prev_nodes = [None for i in range(len(stations))]
prev_nodes[start] = 'start'

dijkstra_time_start = time.time()
result = dijkstra(dist_from_start, start, goal)
pathway = [stations[goal]]
visiting = prev_nodes[goal]
while True:
    pathway.append(stations[visiting])
    if prev_nodes[visiting] == 'start':
        break
    visiting = prev_nodes[visiting]
print('{}から{}までの所要時間は{}分'.format(stations[start], stations[goal], result))
print('{}から{}までの経路は，'.format(stations[start], stations[goal]), list(reversed(pathway)))
dijkstra_time_finish = time.time()
print('プログラムの実行時間：', dijkstra_time_finish - dijkstra_time_start)

渋谷から上野までの所要時間は19分
渋谷から上野までの経路は， ['渋谷', '表参道', '青山一丁目', '永田町', '桜田門', '有楽町', '東京', '神田', '末広町', '上野広小路', '上野']
プログラムの実行時間： 0.0007236003875732422
